## Assessing the Relationship of Target Classes Via ```DBSCAN```  
This notebooks attempts to ascertain if there exists a latent number of classes in the target variable.
This analysis has two primary motivations:
1. If there are clearly less than 4 classes the overall complexity of the classification problem can be reduced.
2. If it is determined that classes within the target have close relationships then the interpretation of the results
and the recommendations could be impacted.

### Packages

In [336]:
import pandas as pd
import numpy as np
import sys
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, TargetEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, root_mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.cluster import DBSCAN
import hdbscan.validity as dbcv_hdbscan
from sklearn.neighbors import KDTree
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.base import BaseEstimator, ClusterMixin
# from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import time
from pathlib import Path
import os

import warnings

# Ignore invalid value encountered in scalar divide warning
warnings.filterwarnings("ignore", category=RuntimeWarning, message="invalid value encountered in scalar divide")

### Set Up ```sys``` Path to Enable ```.py``` Imports

In [330]:
path = Path.cwd()
path_to_project_directory = path.parent
sys.path.insert(1, str(path_to_project_directory))
print(f"The working directory has been set to: {str(path_to_project_directory)}")

The working directory has been set to: /Users/nelsonfarrell/Documents/Northeastern/5220/final_project


In [331]:
from modules.phase1_utils import * 

### Helper Functions

In [332]:
def grid_search_DBSCAN(cap_x, dim_reduction_algo = None, eps_list = [0.3, 0.5, 0.6, 0.75, 0.85, 1.0], min_samples_list = [5, 10, 20, 50, 100], distance_metric = "euclidean"):
    """
    Performs GridSearch over DBSCAN hypers

    Returns:
        * results_dict
    """
    # Results container
    results_row_dict_list = []

    # Hypers
    eps_list = eps_list
    min_samples_list = min_samples_list

    # Get the Hopkins stat to assess cluster propensity
    hopkins = get_hopkins(cap_x)

    # Search loops
    for eps in eps_list:
        for min_sample in min_samples_list:

            # This is used for validity index
            dist_matrix = pairwise_distances(cap_x, metric = distance_metric)
            dist_matrix = dist_matrix.astype(np.float64)

            # Fit DBSCAN
            dbscan = DBSCAN(
                        eps = eps,
                        min_samples = min_sample,
                        metric = distance_metric,
                        metric_params = None,
                        algorithm = "auto",
                        leaf_size = 30,
                        p = None,
                        n_jobs = None
                    )
            dbscan.fit(cap_x)

            # This is for results tracking
            clusters = np.unique(dbscan.labels_)
            n_clusters = clusters[clusters != -1].shape[0]

            # Compute validity index
            try:
                validity_index = dbcv_hdbscan.validity_index(X = dist_matrix, d = cap_x.shape[1] ,labels = dbscan.labels_, metric = 'precomputed')
            except ValueError as e:
                validity_index = np.nan

            # Update results
            results_row_dict_list.append({
                        "n_components": cap_x.shape[1],
                        "n_clusters": n_clusters,
                        'eps': eps,
                        "min_samples": min_sample,
                        "validity_index": validity_index,
                        "hopkins_stat": hopkins,
                        "dim_reduction_algo": dim_reduction_algo
                    })

    return results_row_dict_list

def grid_search_PCA(transformed_data, n_components = [2,3,4,5,6,8,10,15]):
    """
    Iterates over PCA embeddings before DBSCAN gridsearch 
    """
    results_list = []
    n_components = n_components
    for n in n_components:
        transfored_data_reduced = PCA(n_components = n).fit_transform(transformed_data)
        results_row_dict_list = grid_search_DBSCAN(transfored_data_reduced)
        best_results = get_best_results(results_row_dict_list)
        results_list.append(best_results)
    return results_list

def get_hopkins(cap_x):
    '''
    Calculates hopkin's statistic for a design matrix. Measures clustering propensity.
        cap_h = sum(cap_x_nn_dist_list) / (sum(randomly_nn_dist_list) + sum(cap_x_nn_dist_list))

    Parameters:
        * cap_x (np.ndarray): design matrix
    Returns:
        * cap_h (float): hopkin's statistic value
    '''
    # seed random
    np.random.seed(18)

    # get uniformly randomley distributed data
    data_max = cap_x.max(axis=0)
    data_min = cap_x.min(axis=0)
    random_dist_data = np.random.uniform(low=data_min, high=data_max, size=cap_x.shape)

    ## null hypothesis: get nearest neighbor distance (random data)

    # get list of nearest neighbors for random data
    randomly_nn_dist_list = get_NN(random_dist_data)

    # get nearest neighbor distance from embedding
    cap_x_nn_dist_list = get_NN(cap_x)

    # calculate hopkins
    cap_h = sum(cap_x_nn_dist_list) / (sum(randomly_nn_dist_list) + sum(cap_x_nn_dist_list))

    return cap_h

def get_NN(cap_x):
    '''
    Returns nearest neighbors distances list.

    Parameters:
        * cap_x (np.ndarray): design matrix
    Returns:
        * nn_dist_list (list): list of nearest neighbors
            
    Documentation:
        https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KDTree.html
    '''
    # build the kdtree
    kdt = KDTree(cap_x)

    nn_dist_list = []
    for i in range(cap_x.shape[0]):
        dist, indices = kdt.query(cap_x[i, :].reshape(1, -1), 2)
        nn_dist_list.append(dist[0, -1])

    return nn_dist_list

def get_best_results(results_row_dict_list):
    results_df = pd.DataFrame(results_row_dict_list)
    max_index = results_df["validity_index"].idxmax()
    return results_df.loc[max_index]

### Parameters

In [333]:
path_to_data_directory = "/data/data_splits/"
train_data_file_name = "train_df.csv"
target_attr = "Segmentation"
missingness_threshold = 0.20
nominal_imputer_strategy = "most_frequent"

### Set Up Timer

In [ ]:
start = time.time()

___
### Read In Train Data

In [334]:
orig_df = pd.read_csv(str(path_to_project_directory) + path_to_data_directory + train_data_file_name)
train_df = orig_df.copy()
del orig_df

### Assess Target Missingness

In [356]:
num_rows_train_df_pre = train_df.shape[0]
print(f"The shape of train set PRE to dropping rows where the target missing: {train_df.shape}")
train_df = train_df.dropna(subset = target_attr)
num_rows_train_df_post = train_df.shape[0]
print(f"The shape of train set POST to dropping rows where the target missing: {train_df.shape}")
print(f"Number of rows dropped: {num_rows_train_df_pre - num_rows_train_df_post}")

The shape of train set PRE to dropping rows where the target missing: (6454, 12)
The shape of train set POST to dropping rows where the target missing: (6454, 12)
Number of rows dropped: 0


___
### Identify Attributes Above ```Missingness``` Threshold

In [307]:
missingness_results_dict = get_missingness(train_df, missingness_threshold)

index missingness = 0.0
ID missingness = 0.0
Gender missingness = 0.0
Ever_Married missingness = 0.017198636504493336
Age missingness = 0.0
Graduated missingness = 0.00914161760148745
Profession missingness = 0.01642392314843508
Work_Experience missingness = 0.10009296560272699
Spending_Score missingness = 0.0
Family_Size missingness = 0.040904865199876045
Var_1 missingness = 0.009296560272699102
Segmentation missingness = 0.0

missingness_drop_list:
[]


___
### Set Up ML Attributes

In [308]:
non_ml_attributes_results = separate_unique_columns(train_df)

*****************************
non_ML_attr:
index
ID

*****************************
ML_attr:
Gender
Ever_Married
Age
Graduated
Profession
Work_Experience
Spending_Score
Family_Size
Var_1
Segmentation

*****************************
non_ml_attribute_list:
['index', 'ID']


In [309]:
ml_attributes_drop_list = [target_attr] # Drop the target to perform clustering

In [310]:
ml_ignore_list = missingness_results_dict["missingness_drop_list"] \
                 + non_ml_attributes_results["non_ML_attr"] \
                 + ml_attributes_drop_list

print(f"These attributes will be ignored by the machine learning algorithm:")
for idx, attr in enumerate(ml_ignore_list):
    print(f"\t{idx + 1}. {attr}")

These attributes will be ignored by the machine learning algorithm:
	1. index
	2. ID
	3. Segmentation


In [311]:
# Set the numerical attributes list
numerical_attr = ["Age", "Work_Experience", "Family_Size"]

# Set the nominal attributes list
nominal_attr = [attr for attr in train_df.columns if attr not in numerical_attr and attr not in ml_ignore_list]

# Confirm all attributes are accounted for
assert (train_df.shape[1] == len(numerical_attr) + len(nominal_attr) + len(ml_ignore_list))
print(f"All attributes in the train set have been accounted for.")
print()

print("ML Ignore List:")
for idx, attr in enumerate(ml_ignore_list):
    print(f"{idx + 1}. {attr}")
print()

print("Numerical Attributes:")
for idx, attr in enumerate(numerical_attr):
    print(f"{idx + 1}. {attr}")
print()

print("Nominal Attributes:")
for idx, attr in enumerate(nominal_attr):
    print(f"{idx + 1}. {attr}")
print()

print(f"Total Number of ML Attributes: {len(nominal_attr) + len(numerical_attr)}")

All attributes in the train set have been accounted for.

ML Ignore List:
1. index
2. ID
3. Segmentation

Numerical Attributes:
1. Age
2. Work_Experience
3. Family_Size

Nominal Attributes:
1. Gender
2. Ever_Married
3. Graduated
4. Profession
5. Spending_Score
6. Var_1

Total Number of ML Attributes: 9


___
### Inspect the Cardinality of Nominal Attributes  

This will inform the choice of nominal attribute encoder.  

```TargetEncoding``` can reduce the number of resultant encoded attributes compared to ```OneHotEncoding```.  
This becomes an issue when the nominal attributes have a large number of unique values.  

However, in the case of a ```multiclass``` target, ```TargetEncoder``` uses 1 versus all  
approach to computing the probability estimates.  

Resultingly, this may not result fewer dimensions than the
```OneHotEncoder```.

Both encoders will be explored in the context of clustering with ```DBSCAN```.

In [312]:
nominal_cols_df = train_df[nominal_attr]
nominal_cols_df.nunique()

Gender            2
Ever_Married      2
Graduated         2
Profession        9
Spending_Score    3
Var_1             7
dtype: int64

In [347]:
print(f"Number of Nominal Encoded Features Post OneHotEncoding:")
print(f"\tIf dropping one col per attribute: {nominal_cols_df.nunique().sum() - len(nominal_attr)}")
print(f"\tIf NOT dropping one col per attribute: {nominal_cols_df.nunique().sum() + len(numerical_attr)}")

Number of Nominal Encoded Features Post OneHotEncoding:
	If dropping one col per attribute: 19
	If NOT dropping one col per attribute: 28


___
## **OneHotEncoder**

### Data Transformers

In [314]:
numerical_transformer = Pipeline(
                            steps = [("imputer", SimpleImputer()),
                                     ("scaler", StandardScaler())]
                                )

In [315]:
nominal_transformer = Pipeline(
                        steps = [("imputer", SimpleImputer(strategy = nominal_imputer_strategy)),
                                 ("ohe", OneHotEncoder())] 
                              )

In [316]:
preprocessor = ColumnTransformer(
                        transformers = [("nominal", nominal_transformer, nominal_attr),
                                        ("numerical", numerical_transformer, numerical_attr)
                                ]
                        )

### Inspect Transformed Data ~ OneHotEncoder
This shows the output of the data transformations

In [317]:
transfored_data = preprocessor.fit_transform(train_df)

In [318]:
pd.DataFrame(transfored_data)

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.695320,1.942754,-1.227022
1,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.703982,0.000000,-0.560068
2,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.635337,-0.513120,0.773838
3,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.264401,-0.820105,2.107745
4,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.935250,1.942754,-1.227022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6449,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.995233,1.635769,-1.227022
6450,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.395407,-0.820105,-0.560068
6451,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.995233,1.021801,2.107745
6452,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.055215,-0.820105,-0.560068


In [349]:
print(f'Number of attrinutes after OneHotEncoding: {transfored_data.shape[1] + 1}')

Number of attrinutes after OneHotEncoding: 28


### Perform Gridsearch over ```DBSCAN``` with ```OneHotEncoder```

In [350]:
results_row_dict_list = grid_search_DBSCAN(transfored_data)

In [320]:
best_results = get_best_results(results_row_dict_list)

In [321]:
pd.DataFrame(best_results).T

,n_components,n_clusters,eps,min_samples,validity_index,hopkins_stat
20,28.0,140.0,0.85,5.0,0.297498,0.241446


### Dimensionality Reduction with ```PCA``` ~ ```OneHotEncoding```
This cell performs a gridsearch over DBSCAN hypers after dimenstionality reduction with PCA.  

In [322]:
results_list = []
n_components = [2,3,4,5,6,8,10,15]
for n in n_components:
    transfored_data_reduced = PCA(n_components = n).fit_transform(transfored_data)
    results_row_dict_list = grid_search_DBSCAN(transfored_data_reduced)
    best_results = get_best_results(results_row_dict_list)
    results_list.append(best_results)

In [323]:
pd.DataFrame(results_list)

,n_components,n_clusters,eps,min_samples,validity_index,hopkins_stat
3,2.0,2.0,0.30,50.0,-0.563148,0.385945
16,3.0,2.0,0.75,10.0,-0.007134,0.291766
3,4.0,5.0,0.30,50.0,0.035080,0.251862
14,5.0,5.0,0.60,100.0,0.078419,0.221127
0,6.0,175.0,0.30,5.0,0.086437,0.201093


___
## **Target Encoder**

### Adjust Nominal Transformer ~ ```TargetEncoder```

In [341]:
# Swap out OneHotEncoder() for TargetEncoder()
nominal_transformer = Pipeline(
                        steps = [("imputer", SimpleImputer(strategy = nominal_imputer_strategy)),
                                 ("te", TargetEncoder(target_type = "multiclass"))]
                              )

In [342]:
# Update preprocessor
preprocessor = ColumnTransformer(
                        transformers = [("nominal", nominal_transformer, nominal_attr),
                                        ("numerical", numerical_transformer, numerical_attr)
                                ]
                        )

### Inspect Transformed Data ~ ```TargetEncoder```

In [343]:
transfored_data = preprocessor.fit_transform(train_df, train_df[target_attr])

In [344]:
pd.DataFrame(transfored_data)

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,0.243595,0.236872,0.254930,0.264601,0.236035,0.154651,0.118050,0.491204,0.248298,0.265294,...,0.188273,0.132800,0.405341,0.231111,0.236671,0.286454,0.245762,-0.695320,1.942754,-1.227022
1,0.245147,0.224661,0.235086,0.295106,0.250079,0.281160,0.329089,0.139645,0.248298,0.265294,...,0.285662,0.462617,0.068626,0.231111,0.236671,0.286454,0.245762,1.703982,0.000000,-0.560068
2,0.246018,0.234701,0.253558,0.265723,0.241397,0.143910,0.124317,0.490316,0.246420,0.267426,...,0.181423,0.138163,0.408925,0.227237,0.236956,0.289405,0.246399,-0.635337,-0.513120,0.773838
3,0.243934,0.236268,0.254997,0.264800,0.241503,0.287127,0.326288,0.145058,0.243028,0.266001,...,0.296437,0.452701,0.080213,0.232637,0.234702,0.286123,0.246536,0.264401,-0.820105,2.107745
4,0.243595,0.236872,0.254930,0.264601,0.250079,0.281160,0.329089,0.139645,0.237950,0.171616,...,0.188273,0.132800,0.405341,0.251970,0.235727,0.219697,0.292587,-0.935250,1.942754,-1.227022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6449,0.243595,0.236872,0.254930,0.264601,0.236035,0.154651,0.118050,0.491204,0.237950,0.171616,...,0.188273,0.132800,0.405341,0.231111,0.236671,0.286454,0.245762,-0.995233,1.635769,-1.227022
6450,0.243069,0.226505,0.236229,0.294198,0.241397,0.143910,0.124317,0.490316,0.246420,0.267426,...,0.181423,0.138163,0.408925,0.227237,0.236956,0.289405,0.246399,-0.395407,-0.820105,-0.560068
6451,0.243934,0.236268,0.254997,0.264800,0.249274,0.145159,0.122021,0.483486,0.247367,0.168447,...,0.182182,0.135523,0.400019,0.307809,0.228779,0.087867,0.375382,-0.995233,1.021801,2.107745
6452,0.243069,0.226505,0.236229,0.294198,0.241397,0.143910,0.124317,0.490316,0.240959,0.165452,...,0.181423,0.138163,0.408925,0.227237,0.236956,0.289405,0.246399,-1.055215,-0.820105,-0.560068


In [ ]:
print(f"The number of attributes after TargerEncoding: {transfored_data.shape[1] + 1}")

### Perform Gridsearch over ```DBSCAN``` with ```TargetEncoder```

In [351]:
results_row_dict_list = grid_search_DBSCAN(transfored_data)

In [352]:
best_results = get_best_results(results_row_dict_list)

In [353]:
pd.DataFrame(best_results).T

,n_components,n_clusters,eps,min_samples,validity_index,hopkins_stat
0,27.0,187.0,0.3,5.0,0.143158,0.29256


### Dimensionality Reduction with ```PCA``` ~ ```TargetEncoder```  
This cell performs a gridsearch over DBSCAN hypers after dimenstionality reduction with PCA.  

In [354]:
results_list = []
n_components = [2,3,4,5,6,8,10,15]
for n in n_components:
    transfored_data_reduced = PCA(n_components = n).fit_transform(transfored_data)
    results_row_dict_list = grid_search_DBSCAN(transfored_data_reduced)
    best_results = get_best_results(results_row_dict_list)
    results_list.append(best_results)

/Users/nelsonfarrell/miniconda3/envs/conda_usml_env/lib/python3.12/site-packages/hdbscan/validity.py:33: RuntimeWarning: invalid value encountered in divide
  result /= distance_matrix.shape[0] - 1
/Users/nelsonfarrell/miniconda3/envs/conda_usml_env/lib/python3.12/site-packages/hdbscan/validity.py:33: RuntimeWarning: invalid value encountered in divide
  result /= distance_matrix.shape[0] - 1


In [355]:
pd.DataFrame(results_list)

,n_components,n_clusters,eps,min_samples,validity_index,hopkins_stat
0,2.0,2.0,0.3,5.0,-0.260176,0.282965
2,3.0,11.0,0.3,20.0,0.297799,0.139627
9,4.0,7.0,0.5,100.0,0.041375,0.186074
14,5.0,6.0,0.6,100.0,0.043349,0.187294
3,6.0,8.0,0.3,50.0,0.049009,0.195492
